In [ ]:
# 14 April 2023
# nrobot
# acquire urls 

import pandas as pd
from pandarallel import pandarallel
from collections import Counter
pandarallel.initialize(progress_bar=True)
import os
import datetime
import tldextract

In [ ]:
from urlextract import URLExtract
from urllib.parse import urlparse
import os

In [ ]:
# CODE SOURCE: from parse_links import parse_domains
# TODO: consider replacing urlparse with tldextract ?
# TODO: push check_dns flag back to original src file parse_links.py

extractor = URLExtract()
extractor.add_enclosure("(", ")")
extractor.add_enclosure("[", "]")

def parse_domain(url):
    if "//" not in url:
        url = "//" + url
    return urlparse(url).netloc

def parse_domains(text):
    return list(set(parse_domain(url) for url in extractor.gen_urls(text)))

# TODO: refactor the below
def parse_domains_and_validate(text):
    return list(set(parse_domain(url) for url in extractor.gen_urls(text, check_dns=True)))

In [ ]:
print(extractor.get_enclosures())
print(extractor.get_stop_chars_left())
print(extractor.get_stop_chars_right())

In [ ]:
file = './nogit_data/list_of_post_contents.csv'
df = pd.read_csv(file)#, nrows=10_00)

In [ ]:
df.iloc[273225].post_text

In [ ]:
df.sample(10).post_text.dropna().apply(parse_domains)

In [ ]:
# TODO: decorator that annotates filename with pandas version
def retrieve_domains(df, outfile='nogit_data/domains.pkl', overwrite_existing=False):
    # df - must contain 'post_text' column

    # if the file exists, load it.  # if overwrite, then re-run and save it.
    # if file not exists, run and save it.
    # return file
    
    if (not os.path.exists(outfile)) or (os.path.exists(outfile) & overwrite_existing):
        domains = df['post_text'].dropna().parallel_apply(parse_domains)
        domains[domains.apply(bool)].to_pickle(outfile)
    else: 
        domains = pd.read_pickle(outfile)
    
    # TODO: these 3 lines can definitely be one line. 
    t = os.path.getmtime(outfile)
    t = datetime.datetime.fromtimestamp(t)
    t = t.strftime('%H:%M:%S / %B %d %Y')
    print(f'Loaded {outfile}, modified at {t}')
    return domains

def retrieve_domains_and_validate(df, outfile='nogit_data/domains_validated_only.pkl', overwrite_existing=False):
    # same as above, but with check_dns = True on the parse_domain function
    
    if (not os.path.exists(outfile)) or (os.path.exists(outfile) & overwrite_existing):
        domains = df['post_text'].dropna().parallel_apply(parse_domains_and_validate)
        domains[domains.apply(bool)].to_pickle(outfile)
    else: 
        domains = pd.read_pickle(outfile)
    return domains

In [ ]:
df.columns

In [ ]:
from pathlib import Path
DATA_PATH = Path('nogit_data/Case_2')
tag = ''
df = pd.read_pickle(Path(DATA_PATH,f'{tag}preprocessed_df.pd-1.5.1.pkl'))
posts = df[['posted_date_datetime', 'post_text']].copy()
posts['domains'] = retrieve_domains_and_validate(df)



In [ ]:
def count_urls(df):
    # Input: df where each row is a python list of urls

    cnt = Counter()
    all_links = []
    for link_list in df.values:
        all_links.extend(link_list)
    for link in all_links: 
        cnt[link] += 1

    df_links = pd.DataFrame.from_dict(cnt, orient='index').reset_index()
    df_links.columns = ['url', 'counts']
    df_links['normalized_counts'] = df_links['counts'] / sum(df_links['counts']) * 100
    return df_links

def extract_domain(list_url):
    return ['.'.join(tldextract.extract(url)[1:]) for url in list_url]



In [ ]:
posts.domains.dropna()

In [ ]:
urls

In [ ]:
years = [(2018, 2020), (2021, 2023)]
for year0, year1 in years:
    print('year', year0, year1)
    date_filter = (posts.posted_date_datetime >= f'{year0}-01-01') \
        & (posts.posted_date_datetime <= f'{year1}-12-31')
    domains = posts[date_filter].domains.dropna()
    domains = domains.apply(extract_domain)
    counts = count_urls(domains)
    display(counts.sort_values(by='counts', ascending=False).iloc[:20])



In [ ]:
domains_validated_only = retrieve_domains_and_validate(df)

In [ ]:
display(domains.sample(n=40))
domains_validated_only.sample(n=40)

In [ ]:
# TODO - fix url parsing, bug example:
print(domains.loc[111371])
print(domains_validated_only.loc[111371])
#print(df.iloc[111371].post_text)

In [ ]:
def count_urls(df):
    # Input: df where each row is a python list of urls

    cnt = Counter()
    all_links = []
    for link_list in df.values:
        all_links.extend(link_list)
    for link in all_links: 
        cnt[link] += 1

    df_links = pd.DataFrame.from_dict(cnt, orient='index').reset_index()
    df_links.columns = ['url', 'counts']
    df_links['normalized_counts'] = df_links['counts'] / sum(df_links['counts']) * 100
    return df_links

df_links = count_urls(domains)
display(df_links.sort_values(by='counts', ascending=False))
df_links.sort_values(by='counts').iloc[:20]

In [ ]:
# apply to df_links, ? then count again?  

In [ ]:
def extract_domain(list_url):
    return ['.'.join(tldextract.extract(url)[1:]) for url in list_url]
no_suffix = domains.apply(extract_domain)
display(no_suffix)
tmp = count_urls(no_suffix)
tmp
# -- 
no_suffix = domains_validated_only.apply(extract_domain)
display(no_suffix)
tmp = count_urls(no_suffix)
tmp

In [ ]:
tmp[tmp.url == 'twitter.com']

In [ ]:
display(tmp.sort_values(by='counts').iloc[:20])
display(tmp.sort_values(by='counts').iloc[-20:])

In [ ]:

import altair as alt
#tmp.sort_values(by='counts').iloc[-20:].counts.plot(kind='bar', xlabel='url')
_tmp = count_urls(no_suffix)
top20 = _tmp.sort_values(by='counts').iloc[-20:]
chart = alt.Chart(top20).mark_bar(size=20).encode(
    x=alt.X('url', axis=alt.Axis(labelAngle=-30)).sort('-y'),
    y=alt.Y('normalized_counts:Q', title='% of all URLs'),
    text='url'
).properties(width=600, height=200, title='Most Popular URLs (by domain)')
display(chart)
display(top20.iloc[::-1])

In [ ]:
alt.__version__ 
SCALE=1.5
# TODO: refactor into forum_utils.py
def theme_1(*args, **kwargs):
    return {'width': 400, 'height': 300,
            'config': {#'style': {'bar': {'size': 20}},
                       #'legend': {'symbolSize': 20, 'titleFontSize': 20, 'labelFontSize': 20}, 
                       'title':{'fontSize': 20*SCALE},
                       'axis': {'titleFontSize': 15*SCALE, 'labelFontSize': 11*SCALE},
                        "axisX": { #https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602
                                "tickSize": 10, # default, including it just to show you can change it
                                "tickWidth":2,
                        }
            },
    }
            #'encoding': {'x': {'axis': {'title': 'options'}, 'scale': {'paddingOuter': 0.5, 'paddingInner': 0.5}},
            #             'y': {'axis': {'title': 'percentage of students'}}}}
alt.themes.register('theme_1', theme_1)
alt.themes.enable('theme_1')

In [ ]:
import importlib
import forum_utils
importlib.reload(forum_utils)
from forum_utils import get_project_constants, about_utils 

latex_figs = get_project_constants(var='latex_figs')
git_home = get_project_constants(var='git_home')
latex_home = get_project_constants(var='latex_home')
about_utils()

In [ ]:
from pathlib import Path
import altair as alt
importlib.reload(alt)
#tmp.sort_values(by='counts').iloc[-20:].counts.plot(kind='bar', xlabel='url')
_tmp = count_urls(no_suffix)
top20 = _tmp.sort_values(by='counts').iloc[-20:]

base = alt.Chart(top20).encode(
    #x=alt.X('url', axis=alt.Axis(labelAngle=-30), title='Domain').sort('-y'),
    x=alt.X('url', axis=alt.Axis(labelAngle=-30), title='Domain',
    sort=alt.EncodingSortField('normalized_counts', op='min', order='descending')),
    #sort=alt.EncodingSortField(field='y:Q', op='min')),
    y=alt.Y('normalized_counts:Q', title='% of all URLs', scale=alt.Scale(domain=[0,7], clamp=True)),
    text='counts:Q'
).mark_bar()
chart = base.mark_bar(size=20).properties(width=650, height=200, title='20 Most Popular Domains') \
    + base.mark_text(dy=-5) 

#display(chart)

#display(top20.iloc[::-1])
# top20 + base.mark_text(align='left', dx=2)
# First install NodeJS either by direct download or via a package manager, and then use the npm tool to install the required packages:
# $ npm install vega-lite vega-cli canvas
import altair_saver
importlib.reload(altair_saver)
chart = chart.configure(background='#F9F3DC').configure_view(fill='white')
chart.save( str(Path(git_home, latex_figs, 'top_domains.pdf')),engine="altair_saver")
# https://stackoverflow.com/questions/66896225/altair-setting-raw-color-values-messes-up-sortfield
# NOTE: if just use sort('y') get WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
# solution : use encodingsortfield with op 'sum' instead

chart

In [ ]:
top20.iloc[::-1].to_numpy()

In [ ]:
import numpy as np
np.round(2.32342344, 2)

In [ ]:
import pytextable

top20['normalized_counts'] = top20.normalized_counts.apply(lambda x: np.round(x,2))
pytextable.write(top20.iloc[::-1].to_numpy(), str(Path(git_home, latex_home, 'top_domains.tex')))

display(top20.iloc[::-1])

with open(Path(git_home, latex_home, 'top_domains.tex'), 'r') as f:
    print(f.readlines())

In [ ]:
str(Path(git_home, latex_fig, 'chart.pdf'))

In [ ]:
chart = alt.Chart(_tmp).mark_bar(size=20).encode(
    x=alt.X('url', axis=alt.Axis(labelAngle=-30), sort=alt.EncodingSortField(field='-y')),
    y=alt.Y('normalized_counts:Q', title='% of all URLs', scale=alt.Scale(domain=[0,7], clamp=True)),
    text='url'
).properties(width=600, height=200, title='Most Popular URLs (by domain)').configure(background='#F9F3DC').configure_view(fill='white')
display(chart)
display(top20.iloc[::-1])

In [ ]:
display(_tmp[_tmp.url == 'twitter.com'])
_tmp[_tmp.url == 't.co']